# Multi-Agent with OpenAI

In [ ]:
import os
from databricks.connect import DatabricksSession

# if running on a local IDE
spark = DatabricksSession.builder.remote(serverless=True).getOrCreate()

In [ ]:
import os
import mlflow

# TODO make sure you update the config file before this

configs = mlflow.models.ModelConfig(development_config="./config.yaml")
databricks_configs = configs.get("databricks")
agent_configs = configs.get("agent")

CATALOG = databricks_configs["catalog"]
SCHEMA = databricks_configs["schema"]
UC_MODEL = databricks_configs["model"]
WORKSPACE_URL = databricks_configs["workspace_url"]
SQL_WAREHOUSE_ID = databricks_configs["sql_warehouse_id"]
MLFLOW_EXPERIMENT_ID = databricks_configs["mlflow_experiment_id"]
AGENT_NAME = agent_configs["name"]

In [ ]:
import mlflow

mlflow.set_registry_uri("databricks-uc")
mlflow.set_tracking_uri("databricks")

try:
    experiment = mlflow.get_experiment(experiment_id=MLFLOW_EXPERIMENT_ID)
    mlflow.set_experiment(experiment_id=MLFLOW_EXPERIMENT_ID)
    print(f"Set to existing experiment:\n")
    print(experiment)
except mlflow.exceptions.RestException as e:
    if "does not exist" in str(e):
        print(f"Experiment not found. Must create one first.")
    else:
        raise e

## Load & test agent

Make sure you go to the MLflow experiment to look at trace data as you develop & debug the agent.

In [ ]:
%run ./agent.py

In [ ]:
sample_questions = [
    # "Compare and contrast the annual net income growth in the past 10 years between BAC and AXP",
    "What's the debt-to-asset ratio for American Express in the past 10 years, compare to that of Bank of America?",
]

input_example = {
    "input": [
        {
            "role": "user",
            "content": sample_questions[0],
        }
    ]
}

In [ ]:
result = AGENT.predict(input_example)

print(result.model_dump()["output"][-1]["content"][0]["text"])

In [ ]:
for event in AGENT.predict_stream(input_example):
    print(event, "-----------\n")